In [19]:
import numpy as np
import numpy.ma as ma
from astropy.table import Table
import matplotlib.pyplot as plt
from disk_mass import calc_mass_curve, fit_mass_curve
from rotation_fitfunctions import parameterfit_bur, parameterfit_iso, parameterfit_NFW,find_phi, find_axis_ratio, find_incl, chi2
from DRP_rotation_curve import extract_data, extract_Pipe3d_data
from rotation_fitfunctions import find_phi, find_axis_ratio, chi2, find_incl
from disk_mass_plotting_functions import plot_fitted_disk_rot_curve
from disk_mass_functions import chi2_mass
import RC_plotting_functions as RC
%matplotlib inline

In [20]:
full_failed_object_table = Table.read('/scratch/lstroud3/RotationCurves/failed_objects_table.fits')

In [21]:
gal_ID = '10509-1902'
H_0 = 100  # Hubble's Constant in units of h km/s/Mpc
c = 299792.458  # Speed of light in units of km/s

In [22]:
fit_function = 'Isothermal'

In [23]:
MANGA_FOLDER = '/scratch/kdougla7/data/SDSS/dr17/manga/spectro/'
MASS_MAP_FOLDER = MANGA_FOLDER + 'pipe3d/'
VEL_MAP_FOLDER = MANGA_FOLDER + 'analysis/v3_1_1/3.1.0/HYB10-MILESHC-MASTARSSP/'

In [24]:
object_index = {}

for i in range(len(full_failed_object_table)):
    galaxy_ID = full_failed_object_table['plateifu'][i]

    object_index[galaxy_ID] = i


In [47]:
possible_fits = Table(names = ('rhob', 'Rb', 'SigD', 'Rd', 'rho0_h', 'Rh', 'incl', 'phi', 'x_center', 'y_center', 'vsys', 'chi2','fit flag'))

In [33]:
maps = extract_data(VEL_MAP_FOLDER,gal_ID,['Ha_vel', 'r_band', 'Ha_flux', 'Ha_sigma'])
sMass_density, sMass_density_err = extract_Pipe3d_data(MASS_MAP_FOLDER, gal_ID)
i_DRP = object_index[gal_ID]
SN_map = maps['Ha_flux'] * np.sqrt(maps['Ha_flux_ivar'])
vmap_mask = maps['Ha_vel_mask'] + (SN_map < 5)
sM_mask = maps['Ha_vel_mask']
maps['vmasked'] = ma.array(maps['Ha_vel'], mask=vmap_mask)
maps['ivarmasked'] = ma.array(maps['Ha_vel_ivar'], mask=vmap_mask)
maps['rbandmasked'] = ma.array(maps['r_band'], mask = maps['Ha_vel_mask'])
z = full_failed_object_table['nsa_z'][i_DRP]
axis_ratio = full_failed_object_table['nsa_sersic_ba'][i_DRP]
incl = find_incl(axis_ratio)
shape = maps['vmasked'].shape
scale = (0.5 * z * c / H_0) * 1000 / 206265  # kpc
if gal_ID in ['10220-12703','10516-12705','10520-12705','12066-12703',\
                          '11012-12703','8239-12703','8941-12703','8717-12705']:
    center = (37,37)
                
elif gal_ID in ['8466-12705','11021-12702']:
    center = (37,42)
                
elif gal_ID in ['10222-6102','8133-6102']:
    center = (27,27)

elif gal_ID == '11830-3704':
    center = (20,19)         
            
elif gal_ID == '9879-6101':
    center = (30,27)      

elif gal_ID in ['7443-3701','8335-3704','9514-3702','8318-3702','8657-3702','7815-3704','8341-3702']:
    center = (22,22)  

elif gal_ID == '8728-6104':
    center = (25,25)
elif gal_ID in ['10223-12704','10223-12704']:
    center = (41,31)
elif gal_ID == '11945-9101':
    center = (31,31)
            
elif gal_ID == '12066-12703':
    center = (37,30)
else:
            center = np.unravel_index(ma.argmax(maps['rbandmasked']), shape)
x0 = center[0]
y0 = center[1]
rhoh = -1.5
Rh = 10
vsys = 0


In [36]:
phi_starts = []
for i in range(18):
    phi_starts.append(i*10 + 1)
print(phi_starts)

[1, 11, 21, 31, 41, 51, 61, 71, 81, 91, 101, 111, 121, 131, 141, 151, 161, 171]


In [48]:
for phi in phi_starts: 
    fit_flag = 0
    phi_guess = find_phi(center,phi,maps['vmasked'])
    param = [rhoh,Rh,incl,phi_guess,x0,y0,vsys]
    fit = np.ones(11)
    chi2 = None
    for i in range(50):
        rhoh,Rh,incl,phi,x0,y0,vsys = param
        best_fit = None
            ##disk fit
            #print("Fitting disk")
        rhoh,Rh,incl,phi,x0,y0,vsys = param
        axis_ratio = find_axis_ratio(incl)
                
        mass_data_table = calc_mass_curve(sMass_density, sMass_density_err, maps['r_band'], \
                                                  sM_mask, x0, y0, axis_ratio, phi, z, gal_ID)
        if len(mass_data_table)<10:
            fit_flag = -5
            break;
        param_outputs = fit_mass_curve(mass_data_table, gal_ID, 'bulge')
        
        if param_outputs == None:
            fit_flag = -5
            break;
        #total fit
        print("Fitting velocity map", flush=True)
        if fit_function == 'Isothermal':
            best_fit = parameterfit_iso(param, param_outputs['rho_bulge'], param_outputs['R_bulge'],\
                                      param_outputs['Sigma_disk'], param_outputs['R_disk'], scale,\
                                       shape, maps['vmasked'], maps['ivarmasked'], vmap_mask, gal_ID)
        elif fit_function == 'NFW':
            best_fit = parameterfit_NFW(param, param_outputs['rho_bulge'], param_outputs['R_bulge'],\
                                           param_outputs['Sigma_disk'], param_outputs['R_disk'], scale,\
                                           shape, maps['vmasked'], maps['ivarmasked'], vmap_mask, gal_ID)
               
        elif fit_function == 'Burkert':
            best_fit = parameterfit_bur(param, param_outputs['rho_bulge'], param_outputs['R_bulge'],\
                                        param_outputs['Sigma_disk'], param_outputs['R_disk'], scale,\
                                        shape, maps['vmasked'], maps['ivarmasked'], vmap_mask, gal_ID)
        else:
            print("Fit function not known", flush=True)
            
        if best_fit is None:
            print('best fit is none')
            fit_flag = -6
            break;
                
                
        #check if fit is close enough to past iteration
        if np.abs(best_fit[2]-param[2])<0.001 and np.abs(best_fit[4]-param[4])<1 and np.abs(best_fit[3]-param[3])<0.0017 \
                and np.abs(best_fit[5]-param[5])<1 and np.abs(best_fit[6]-param[6])<0.01:
            fit_flag = i
            print("Converged", flush=True)
            break
                
        param = best_fit
    if fit_flag >= 0:
        fit = [param_outputs['rho_bulge'], param_outputs['R_bulge'],\
                                   param_outputs['Sigma_disk'], param_outputs['R_disk'], \
                                   best_fit_fit[0], best_fit[1], best_fit[2], best_fit[3], \
                                   best_fit[4],best_fit[5],best_fit[6]]
        chi2 = chi2(maps['vmasked'], maps['ivarmasked'],vmap_mask,shape,scale,fit,fit_function)[1]
    possible_fits.add_row((fit[0],fit[1],fit[2],fit[3],fit[4],fit[5],fit[6],fit[7],fit[8],fit[9],fit[10],chi2,fit_flag))

1.5827397
(17, 17)
0.017453292519943295
[360  11]
[360  11]
[303  12]
[246  13]
[246  13]
[246  13]
[188  14]
[188  14]
[131  15]
[131  15]
[131  15]
[131  15]
[74 16]
[74 16]
[74 16]
[74 16]
[74 16]
[74 16]
[74 16]
[17 17]
finding mass curve
1.5827397
(17, 17)
0.19198621771937624
[47 11]
[47 11]
[42 12]
[37 13]
[37 13]
[37 13]
[32 14]
[32 14]
[27 15]
finding mass curve
1.5827397
(17, 17)
0.3665191429188092
[32 11]
[32 11]
[30 12]
[27 13]
finding mass curve
1.5827397
(17, 17)
0.5410520681182421
[26 11]
finding mass curve
1.5827397
(17, 17)
0.715584993317675
[23 11]
finding mass curve
1.5827397
(17, 17)
0.890117918517108
[21 11]
finding mass curve
1.5827397
(17, 17)
1.064650843716541
[20 11]
finding mass curve
1.5827397
(17, 17)
1.239183768915974
[19 11]
finding mass curve
1.5827397
(17, 17)
1.413716694115407
[17 11]
finding mass curve
1.5827397
(17, 17)
1.5882496193148399
[17 11]
finding mass curve
1.5827397
(17, 17)
1.7627825445142729
[16 11]
finding mass curve
1.5827397
(17, 17)
1.93

In [49]:
print(possible_fits)

rhob  Rb SigD  Rd rho0_h  Rh incl phi x_center y_center vsys chi2 fit flag
---- --- ---- --- ------ --- ---- --- -------- -------- ---- ---- --------
 1.0 1.0  1.0 1.0    1.0 1.0  1.0 1.0      1.0      1.0  1.0  nan     -5.0
 1.0 1.0  1.0 1.0    1.0 1.0  1.0 1.0      1.0      1.0  1.0  nan     -5.0
 1.0 1.0  1.0 1.0    1.0 1.0  1.0 1.0      1.0      1.0  1.0  nan     -5.0
 1.0 1.0  1.0 1.0    1.0 1.0  1.0 1.0      1.0      1.0  1.0  nan     -5.0
 1.0 1.0  1.0 1.0    1.0 1.0  1.0 1.0      1.0      1.0  1.0  nan     -5.0
 1.0 1.0  1.0 1.0    1.0 1.0  1.0 1.0      1.0      1.0  1.0  nan     -5.0
 1.0 1.0  1.0 1.0    1.0 1.0  1.0 1.0      1.0      1.0  1.0  nan     -5.0
 1.0 1.0  1.0 1.0    1.0 1.0  1.0 1.0      1.0      1.0  1.0  nan     -5.0
 1.0 1.0  1.0 1.0    1.0 1.0  1.0 1.0      1.0      1.0  1.0  nan     -5.0
 1.0 1.0  1.0 1.0    1.0 1.0  1.0 1.0      1.0      1.0  1.0  nan     -5.0
 1.0 1.0  1.0 1.0    1.0 1.0  1.0 1.0      1.0      1.0  1.0  nan     -5.0
 1.0 1.0  1.0 1.0    1.0 